In [17]:
#%%capture

!pip install sentence-transformers


  Using cached sentence_transformers-5.1.0-py3-none-any.whl (483 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 425.4 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.0/888.0 MB 673.8 kB/s eta 0:00:0000:0100:24
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 835.9 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.6/199.6 kB 2.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 1.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.2/287.2 MB 500.8 kB/s eta 0:00:0000:0100:20
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 938.3 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 763.0 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 MB 667.6 kB/s eta 0:00:0000:0100:06
     ╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/706.8 MB 739.3 kB/s eta 0:15:35

In [6]:
import sys
import os

# Añadir la carpeta src al path
sys.path.append(os.path.join("..", "src"))

from data_loader import load_all

In [7]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from openai import OpenAI
from data_loader import load_all

In [9]:
users, products, interactions = load_all(
    "../data/users.csv",
    "../data/products.csv",
    "../data/interactions.csv"
)

# Combinar palabras clave y descripción para texto del producto
products['text_features'] = products.apply(
    lambda row: " ".join(row['palabras_clave']) + " " + str(row['descripcion']),
    axis=1
)

In [ ]:
# Modelo de embeddings ligero
embed_model = SentenceTransformer('all-MiniLM-L6-v2')
product_embeddings = embed_model.encode(products['text_features'].tolist(), convert_to_tensor=True)

In [11]:
client = OpenAI(api_key="sk-proj---PT") 

In [ ]:
product_texts = products['text_features'].tolist()
resp = client.embeddings.create(
    model="text-embedding-3-small",
    input=product_texts
)
product_embeddings = np.array([e.embedding for e in resp.data])


In [ ]:
def recommend_rag(user_id: int, top_k: int = 10):
    if user_id not in users['user_id'].values:
        return "Usuario no encontrado"

    user = users[users['user_id'] == user_id].iloc[0]

    # Embedding de intereses del usuario
    user_text = " ".join(user['intereses'])
    user_emb = embed_model.encode(user_text, convert_to_tensor=True)

    # Recuperación: top-k productos más similares
    hits = util.semantic_search(user_emb, product_embeddings, top_k=top_k)[0]
    top_products = products.iloc[[hit['corpus_id'] for hit in hits]]

    # --------------------------
    # 4️⃣ Generación con LLM
    # --------------------------
    client = OpenAI(api_key="TU_API_KEY_AQUI")  # reemplaza con tu API key

    # Formar prompt para el LLM
    product_info = "\n".join(
        [f"{row['name']} ({row['category']}): {row['text_features']}" for _, row in top_products.iterrows()]
    )

    prompt = f"""
Eres un asistente que recomienda productos de bienestar.
Usuario: edad {user['edad']}, género {user['genero']}, intereses: {', '.join(user['intereses'])}.
Productos filtrados: {product_info}
Sugiere 5 productos relevantes y explica brevemente por qué son útiles.
Devuelve en JSON: [{"product_id": "...", "name": "...", "category": "..."}]
"""

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=500
    )

    return response.choices[0].message.content

In [ ]:
recommend_rag(user_id=1, top_k=10)